In [ ]:
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1

In [ ]:
!pip install einops

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import json
import warnings

import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from tqdm.notebook import tqdm

from trl import SFTTrainer
from huggingface_hub import interpreter_login
os.environ["WANDB_DISABLED"] = "true"
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel("/content/finetuningwhisper.xlsx")
df = df.iloc[:,:2]
df.columns = ["Original Paragraph",'Corrected Paragraph']
df

In [ ]:
df = df[~df['Corrected Paragraph'].isnull()]

In [ ]:
df

In [ ]:
!pip install accelerate
!pip install bitsandbytes

In [ ]:
result = list(df.to_json(orient="records"))

result[0] = '{"json":['
result[-1] = ']'
result.append('}')


result = ''.join(result)
result = result.strip('"\'')

result = json.loads(result)

with open('data.json', 'w') as json_file:
    json.dump(result, json_file)

In [ ]:
result

In [ ]:
import json

# Assuming 'result' is your original JSON data
result = json.loads(result)  # Load it as a Python object if it's a string

new_format = []

for entry in result['json']:
    new_entry = {
        "prompt": entry['Original Paragraph'],
        "completion": entry['Corrected Paragraph']
    }
    new_format.append(new_entry)

new_json = json.dumps(new_format, indent=4)
print(new_json)


In [ ]:
dataset = load_dataset("json", data_files="data.json", field='json', split="train")
def format_instruction(Original_Paragraph: str, Corrected_Paragraph: str):
	return f"""### Instruction:
Correct the paragraph:

### Original Paragraph:
{Original_Paragraph.strip()}

### Corrected Paragraph:
{Corrected_Paragraph}
""".strip()
def process_dataset(data: dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset)
    )


def generate_instruction_dataset(data_point):

    return {
        "Original Paragraph": data_point["Original Paragraph"],
        "Corrected Paragraph": data_point["Corrected Paragraph"],
        "text": format_instruction(data_point["Original Paragraph"],data_point["Corrected Paragraph"])
    }
dataset = process_dataset(dataset)


In [ ]:
dataset

In [ ]:
from peft import prepare_model_for_int8_training
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=False,
)

model_id =  "microsoft/phi-2"
#Download model
model = AutoModelForCausalLM.from_pretrained( model_id, quantization_config=bnb_config, trust_remote_code=True, flash_attn=True, flash_rotary=True, fused_dense=True, low_cpu_mem_usage=True, device_map={"": 0}, revision="refs/pr/23" )

model.config.pretraining_tp = 1
peft_config = LoraConfig(
r=32,
lora_alpha=16,
target_modules=[
'Wqkv',
'out_proj'
],
bias="none",
lora_dropout=0.05,
task_type="CAUSAL_LM",

)

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = prepare_model_for_int8_training(model, use_gradient_checkpointing=True)

training_arguments = TrainingArguments(
output_dir= "./results",
num_train_epochs= 5,
per_device_train_batch_size= 1,
gradient_accumulation_steps= 1,
optim="paged_adamw_32bit",
save_strategy="epoch",
logging_steps=100,
logging_strategy="steps",
learning_rate= 2e-4,
fp16= False,
bf16= False,
group_by_length= True,
disable_tqdm=False,
report_to=None
)
model.config.use_cache = False


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=600,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)
trainer.train()

In [ ]:
!huggingface-cli login

In [ ]:
model.push_to_hub("medical_corrector_phi2",trust_remote_code=True)

In [ ]:
trainer.model.push_to_hub("medical_corrector_phi2",trust_remote_code=True)

In [ ]:
list_ = []

def split_string_on_dot(s):
    # Split the string into a list of strings based on '.' if the length exceeds 900
    curr_len = 0
    curr_str = ''

    for p in s:
        curr_len += 1
        curr_str += p

        if p == '.' and curr_len >= 900:
            list_.append(curr_str)
            curr_len = 0
            curr_str = ''

    # Include the last remaining portion
    if curr_str:
        list_.append(curr_str)

# Example usage

str=" My name is Dr. Hariharumurthy. I've got about three decades of experience as an ENT surgeon, and I'm practicing now with the Apollo Spectra Hospitals, Koramangala. Ear infections commonly are seen in the external ear, outside the ear drum or in the middle ear inside the ear drum. This is the common nature of infections that we come across. Commonly seen in the external ear are infections caused by self cleaning habits for example people have the habit of using different types of instruments which can be pins, pens, keys, ear buds. In fact the buds are not even designed for cleaning the ear. So all these implements that are used to clean the ear is actually an exercise in futility because the ear is a self-cleaning mechanism. When one chews their food, the movement of the external part of the external ear is sufficient to bring out the excessive wax and bring it out without your intervention. Only when we try to perfect what nature has done, we tend to cause problems. And you can have injuries which can cause infections. Besides, you can push the wax inside and injure the structures of the external ear. There have been instances where the ear drum has been ruptured because of energetic self-cleaning because you're not seeing what you're cleaning so these are the things that can cause infections of the external ear swimming in dirty ponds is another cause for infection of the ear or swimmers ear which can be also identified as otomycosis. So these are practices which are not safe. Cleaning of the ear routinely with anything is completely proscribed. Coming to the middle ear infections, infections of the nose or the throat can track upwards into the ear and cause infections in the ear. So if a child is having any infection in the nose or the throat or having a severe cold or in a very small child between two to four years of age or five there can be tonsils and adenoids which can be the harboring areas of infection which easily get driven into the ear blowing your nose very hard is not a very good idea because that can also force bacteria into the middle ear from the nose. So treating the causes in the nose or the throat is a very critical aspect of preventing infections happening in the ear. Young children who have repetitive infections which can happen in the school going age in the early years is a common factor for infections in the year. So these children have to be treated for their primary problem and their immune responses have to be bolstered by proper practices to ensure that the infections don't happen in the year. you"
split_string_on_dot(str)
print(list_)

In [ ]:
# str="hi all I am doctor pulmonary just practicing at Sachar nagar inspired Specialty Clinic Bangalore can flu cause pneumonia definitely most of the time 90% of the time flu influenza can cause some viral infections and it is self-limiting without any treatment also it can improve on its own with symptomatic treatment few patients need antivirals to patients can get secondary bacterial infections especially staphylococcal infections or they can get pneumonias with the streptococcus and flew by makes because of the airway Bridge since it affects the Airways and they are more susceptible to stop OK Google sometimes it can be life threatening and it can cause severe name of the tracks complications media all those things can happen infections and viral infections for sale can cause pneumonia and they may present like bilateral pneumonia which can be life threatening affecting oxygenation and they just require one or two liters of oxygen and some people because of the strange regions they may progressed faster and 7th or 8th day they mean more complications can happen which is called as which requires invisible mechanical ventilation and it can go up to 110% of the complications which can happen with the fluid they make secondary infection sometimes they go for multi organizations and death is very less than 4% of the chances is there that's why we tell them to take vaccinations at the disease will not be seen and they will not die much and the people know about the flu very well and every year flu vaccination is a preventative modality and pneumonia are definitely common with flu and incidents of come down because we have a specific treatment for influenza and we have a vaccinations which can cover for one year that's about flu"
# split_string_on_dot(str)
# list_=[]
corrected_strings = []
for sentence in list_:
    inputs = tokenizer(f"Correct the paragraph.Your role is to analyze statements sentence by sentence. You should not summarize and do not skip any line and instead focus on correcting each sentence individually. Adress any minor grammatical issues and fact-check sentences specifically related to the medical domain.Rely on your own knowledge or provided documents for correcting medical facts.For sentence having no medical facts, check for grammar.If there are corrections return the corrected sentence else return the orignal sentence.Original Paragraph: '{sentence}' Corrected Paragraph:", return_tensors="pt", return_attention_mask=False)
    outputs = model.generate(**inputs, max_length=600)
    text = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)
    corrected_strings.append(''.join(text))
    print(''.join(text))
list_.clear()

In [ ]:
corrected_strings = []

for sentence in list_:
    inputs = tokenizer(f"Correct the paragraph.Your role is to analyze statements sentence by sentence. You should not summarize and do not skip any line and instead focus on correcting each sentence individually. Address any minor grammatical issues and fact-check sentences specifically related to the medical domain. Rely on your own knowledge or provided documents for correcting medical facts. For sentence having no medical facts, check for grammar. If there are corrections return the corrected sentence else return the original sentence. Original Paragraph: '{sentence}'", return_tensors="pt", return_attention_mask=False)
    outputs = model.generate(**inputs, max_length=500)
    generated_text = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)

    # Split the generated text into lines
    lines = ''.join(generated_text).split('\n')

    # Find the line that starts with "Corrected Paragraph:"
    corrected_paragraph_line = next(
        (line for line in lines if line.startswith("Corrected Paragraph:")), None
    )

    if corrected_paragraph_line:
        # Append the part after "Corrected Paragraph:" to the corrected_strings list
        corrected_strings.append(corrected_paragraph_line[len("Corrected Paragraph: "):])

list_.clear()
single_paragraph = ' '.join(corrected_strings)

# Print the result
print(single_paragraph)

In [ ]:
single_paragraph = ' '.join(corrected_strings)

# Print the result
print(single_paragraph)

In [ ]:
len(list_)

In [ ]:
len(list_)

In [ ]:
list_

In [ ]:
# str="hi all I am doctor pulmonary just practicing at Sachar nagar inspired Specialty Clinic Bangalore can flu cause pneumonia definitely most of the time 90% of the time flu influenza can cause some viral infections and it is self-limiting without any treatment also it can improve on its own with symptomatic treatment few patients need antivirals to patients can get secondary bacterial infections especially staphylococcal infections or they can get pneumonias with the streptococcus and flew by makes because of the airway Bridge since it affects the Airways and they are more susceptible to stop OK Google sometimes it can be life threatening and it can cause severe name of the tracks complications media all those things can happen infections and viral infections for sale can cause pneumonia and they may present like bilateral pneumonia which can be life threatening affecting oxygenation and they just require one or two liters of oxygen and some people because of the strange regions they may progressed faster and 7th or 8th day they mean more complications can happen which is called as which requires invisible mechanical ventilation and it can go up to 110% of the complications which can happen with the fluid they make secondary infection sometimes they go for multi organizations and death is very less than 4% of the chances is there that's why we tell them to take vaccinations at the disease will not be seen and they will not die much and the people know about the flu very well and every year flu vaccination is a preventative modality and pneumonia are definitely common with flu and incidents of come down because we have a specific treatment for influenza and we have a vaccinations which can cover for one year that's about flu"
# split_string_on_dot(str)
# list_=[]
corrected_strings = []
# for sentence in list_:
#     inputs = tokenizer(f"Correct the paragraph.Your role is to analyze statements sentence by sentence. You should not summarize and do not skip any line and instead focus on correcting each sentence individually. Adress any minor grammatical issues and fact-check sentences specifically related to the medical domain.Rely on your own knowledge or provided documents for correcting medical facts.For sentence having no medical facts, check for grammar.If there are corrections return the corrected sentence else return the orignal sentence.Original Paragraph: '{sentence}' Corrected Paragraph:", return_tensors="pt", return_attention_mask=False)
#     outputs = model.generate(**inputs, max_length=500)
#     text = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)
#     corrected_strings.append(''.join(text))
#     print(''.join(text))
# list_.clear()
# for sentence in list_:
#     inputs = tokenizer(f"Correct the paragraph. Your role is to analyze statements sentence by sentence. You should not summarize and do not skip any line and instead focus on correcting each sentence individually. Address any minor grammatical issues and fact-check sentences specifically related to the medical domain. Rely on your own knowledge or provided documents for correcting medical facts. For sentence having no medical facts, check for grammar. If there are corrections return the corrected sentence else return the original sentence.- '{sentence}' ", return_tensors="pt", return_attention_mask=False)
#     outputs = model.generate(**inputs, max_length=500)
#     text = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)
#     corrected_text = ''.join(text).split('\n')[0]  # Split the text at '\n' and take the first part
#     corrected_strings.append(corrected_text)
#     print(corrected_text)
# list_.clear()
# corrected_paragraph = ' '.join(corrected_strings)

# Optionally, you can print the entire corrected paragraph
# print(corrected_paragraph)
corrected_strings = []
for sentence in list_:
    inputs = tokenizer(f"Correct the paragraph. Your role is to analyze statements sentence by sentence. You should not summarize and do not skip any line and instead focus on correcting each sentence individually. Address any minor grammatical issues and fact-check sentences specifically related to the medical domain. Rely on your own knowledge or provided documents for correcting medical facts. For sentence having no medical facts, check for grammar. If there are corrections return the corrected sentence else return the original sentence. Original Paragraph: '{sentence}' Corrected Paragraph:", return_tensors="pt", return_attention_mask=False)
    outputs = model.generate(**inputs, max_length=1000)

    # Decode the outputs and join into a single string
    text = ''.join(tokenizer.batch_decode(outputs[0], skip_special_tokens=True))

    # Split the text at "Corrected Paragraph:" and take the part after the first occurrence
    parts = text.split("Corrected Paragraph:")
    if len(parts) > 1:
        corrected_text = parts[1].strip()  # Take the second element (first element after the phrase)
        corrected_strings.append(corrected_text)
        print(corrected_text)

# Joining all corrected sentences into a single paragraph
corrected_paragraph = ' '.join(corrected_strings)

# Optionally, you can print the entire corrected paragraph
cleaned_text = corrected_paragraph.replace("'", "")
list_.clear()


In [ ]:
import pandas as pd


# Function to split long paragraphs into smaller parts
def split_string_on_dot(s):
    list_ = []
    curr_len = 0
    curr_str = ''

    for p in s:
        curr_len += 1
        curr_str += p

        if p == '.' and curr_len >= 900:
            list_.append(curr_str)
            curr_len = 0
            curr_str = ''

    if curr_str:  # Include the last remaining portion
        list_.append(curr_str)

    return list_

# Function to correct a paragraph
def correct_paragraph(sentence, tokenizer, model):
    inputs = tokenizer(f"Correct the paragraph. Your role is to analyze statements sentence by sentence. You should not summarize and do not skip any line and instead focus on correcting each sentence individually. Address any minor grammatical issues and fact-check sentences specifically related to the medical domain. Rely on your own knowledge or provided documents for correcting medical facts. For sentence having no medical facts, check for grammar. If there are corrections return the corrected sentence else return the original sentence. Original Paragraph: '{sentence}' Corrected Paragraph:", return_tensors="pt", return_attention_mask=False)
    outputs = model.generate(**inputs, max_length=1000)
    text = ''.join(tokenizer.batch_decode(outputs[0], skip_special_tokens=True))
    parts = text.split("Corrected Paragraph:")
    return parts[1].strip() if len(parts) > 1 else sentence

# Load the CSV file
df = pd.read_csv('final_test.csv')

corrected_paragraphs = []

# Process only the first paragraph and then break
# for i, row in df.iterrows():
#     original_paragraph = row['Original Paragraph']
#     split_paragraphs = split_string_on_dot(original_paragraph)
#     corrected_parts = [correct_paragraph(part, tokenizer, model) for part in split_paragraphs]
#     corrected_paragraph = ' '.join(corrected_parts).replace("'", "")  # Reassemble and remove quotation marks

#     # Update the dataframe with the corrected paragraph
#     df.at[i, 'Corrected Paragraph'] = corrected_paragraph

#     # Save the file after each iteration
#     intermediate_filename = f'corrected_data_{i+1}.csv'
#     df.to_csv(intermediate_filename, index=False)
#     print(f"Updated file saved: {intermediate_filename}")

#     # Break after processing the first entry for testing
#     if i == 0:
#         break
from google.colab import files
for i, row in df.iloc[1:].iterrows():  # Start from the second row
    original_paragraph = row['Original Paragraph']
    split_paragraphs = split_string_on_dot(original_paragraph)
    corrected_parts = [correct_paragraph(part, tokenizer, model) for part in split_paragraphs]
    corrected_paragraph = ' '.join(corrected_parts).replace("'", "")  # Reassemble and remove quotation marks

    # Update the dataframe with the corrected paragraph
    df.at[i, 'Corrected Paragraph'] = corrected_paragraph
    print(corrected_paragraph)
    # Save and download the file after every 15 or 16 entries
    if (i + 1) % 16 == 0:
        intermediate_filename = f'corrected_data_{i+1}.csv'
        df.to_csv(intermediate_filename, index=False)
        print(f"Intermediate file saved: {intermediate_filename}")
        files.download(intermediate_filename)  # Download the file

# Save the final file and download it
final_filename = 'final_corrected_data.csv'
df.to_csv(final_filename, index=False)
print(f"Final file saved: {final_filename}")
files.download(final_filename)


# # Note: Since we're only running this for one entry, the final file will also contain only this entry
# final_filename = 'final_corrected_data_single_entry.csv'
# df.to_csv(final_filename, index=False)
# print(f"Final file saved: {final_filename}")

In [ ]:
final_filename = 'final_corrected_data.csv'
df.to_csv(final_filename, index=False)
print(f"Final file saved: {final_filename}")


In [ ]:
corrected_strings

In [ ]:
corrected_paragraph


cleaned_text


In [ ]:
inputs = tokenizer('''Correct the paragraph.Your role is to analyze statements sentence by sentence. You must avoid summarizing and instead focus on correcting each sentence individually. Adress any minor grammatical issues and fact-check sentences specifically related to the medical domain.Rely on your own knowledge or provided documents for correcting medical facts.For sentence having no medical facts, check for grammar.If there are corrections return the corrected sentence else return the orignal sentence.
Original Paragraph: [Unrecognized speech] hello everyone my name is Dr Dheeraj Kumar Tiwari in Narayana health today we will be discussing about the benign thyroid nodules so benign thyroid nodules are the non cancerous over growth in the thy Chhotu the calls because cosmetic problem sometimes you will see swelling over here if they grow very big then they may compress the air so they can cause difficult in breathing and sometimes they can over produce the hor so these are the common problems with the benign thyroid so what are the treatment options for benign thyroid nodules the treatment options are surgical and non surgical surgery has been done for quite some time but the biggest disadvantage of surgery is that it leaves his car over the neck one thing second the pain is the total recovery time is and if it damages the nerves then it can give some impact on the voice the latest advancement in the thyroid nodules are the ablation in ablation we have multiple types of application like radio frequency evolution and microwave ovulation these are Minimal invasive methods by which these benign thyroid nodules can be treated so if the patient is not willing for Sur if the patient has [Unrecognized speech] cosmetic problem because of thyroid or if the thyroid nodule is causing some compression over the Airway  Corrected Paragraph:''', return_tensors="pt", return_attention_mask=False)
outputs = model.generate(**inputs, max_length=700)
text = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)
print(''.join(text))
# Sometimes we will see swelling over here. If they grow very big then they may compress the airway. So they can cause difficulty in breathing and sometimes they can overproduce the hormones. So these are the common problems with the benign thyroid nodules. So what are the treatment options for benign thyroid nodules? The treatment options are surgical and non-surgical. Surgery has been done for quite some time. But the biggest disadvantage of surgery is that it leaves a scar over the neck. One thing. Second, the pain is more. The total recovery time is far more. And if it damages the nerves, then it can give some impact on the voice. So the latest advancement in the thyroid nodules are the ablation. In ablation we have multiple types of ablation like radio frequency ablation and microwave ablation. These are minimally invasive methods by which these benign thyroid nodules can be treated. So if the patient is not willing for surgery, if the patient has cosmetic problem because of thyroid or if the thyroid nodule is causing some compression over the airway. They are the candidates who should undergo these therapies. How do we do it? So this treatment is done under local anesthesia. The patient is admitted for a day care admission and it's usually a walk in walk out procedure where the patient walks in the hospital and walks out on his own so it is done in local anesthesia so first of all we give an anesthetic injection around the nodule. Then we insert a microwave or radio frequency antenna, which is less than two millimeter in size and send that antenna inside that nodule and then we burn it from inside. So the recovery time is very fast and it usually does not leave any scar and the chances of injury to different organs are very less. And when we burn this nodule, gradually the nodule shrinks in size and the cosmetic problems totally goes away. So if you have any overgrowth in the neck region which is involving thyroid you can always consult your interventional radiologist so that it can be confirmed by doing fine needle aspiration that whether these are benign or malignant. If they are benign, then you can always receive such treatments.

In [ ]:
final_response="Original Paragraph:My name is Dr. Harihar Murthy. I've got about three decades of experience as an ENT surgeon and I'm practicing now with the Apollo Spectra Hospitals, Koramangala. Snoring is a problem that has got multiple reasons and identifying the factors is critical. Obesity is one of them. Obstructions at the level of the nose or nasal pharynx are another. Children with adenoids can also have snoring. Just like adults having obstructions which can cause the problem, children can also have snoring. In the case of adults, usually the problem is because of obstructions like I said along with palatal elongation, flutter of the palate when breathing from the mouth as well as collapse of the pillars of the tonsils or the epic lotus itself can be a flutter causing snoring so identifying the cause is critical dealing with obesity is one of the important factors of snoring treatment. As long as that is done, we also have to address the local factors like doing surgery for the palate. Sometimes we do restructuring of the palate and the pharyngeal tissues. As a result of that we reduce the flutter chances of the elongated and patulous palate. Sometimes we do put sutures in the palate and the pillars of the tonsil which restrict the mobility as a result reduce the snoring. Such are the factors which have to be addressed when dealing with snoring. So a holistic look at the causes for that particular patient would be necessary before we choose the particular treatment which is required for him. Ð ÐµÐ´Ð°ÐºÑ‚Ð¾Ñ€ ÑÑƒÐ±Ñ‚Ð¸Ñ‚Ñ€Ð¾Ð² Ð.Ð¡ÐµÐ¼ÐºÐ¸Ð½ ÐšÐ¾Ñ€Ñ€ÐµÐºÑ‚Ð¾Ñ€ Ð.Ð•Ð³Ð¾Ñ€Ð¾Ð²Ð°"
response="Corrected Paragraph: My name is Dr. Harihara Murthy. I have about three decades of experience as an ENT surgeon and I'm currently practicing at Apollo Spectra Hospitals, Koramangala. Snoring is a problem that has multiple reasons, and identifying the factors is critical. Obesity is one of them. Obstructions at the level of the nose or nasal pharynx are another. Children with adenoids can also have snoring. Just like adults having obstructions that can cause the problem, children can also have snoring. In the case of adults, usually the problem is because of obstructions, like I said along with palatal elongation, flutter of the palate when breathing from the mouth, and collapse of the pillars of the tonsils or the epic lotus itself can be a flutter causing snoring, identifying the cause is critical. Dealing with obesity is one of the important factors of snoring treatment. As long as that is done, the problem can we also have to address the local factors like surgery for the palate. Sometimes we do restructuring of the palate and the pharyngeal tissues. As a result of that we reduce the flutter chances of the elongated and patulous palate. Sometimes we do put sutures in the palate and the pillars of the tonsil which restrict the mobility as a result reduce the snoring. Such are the factors which have to be addressed when dealing with snoring. So a holistic look at the causes for that particular patient would be necessary before we choose the particular treatment which is required for him. Thank you."

from IPython.display import display, HTML
html_content = f'<pre style="white-space: pre-wrap; font-family: monospace;">{final_response}</pre>'
st= f'<pre style="white-space: pre-wrap; font-family: monospace;">{response}</pre>'
# Display the HTML content
# print(response)
display(HTML(html_content))
display(HTML(st))